# 📊 Fase 01: Data Discovery & Extraction
Este notebook se encarga de la extracción de datos desde Supabase utilizando la API REST y realiza una auditoría detallada de 13 puntos de calidad.

In [1]:
# Celda 1: Importacion de librerias y Parche de Codificacion Inteligente
import sys
import os
import builtins
import json
import pandas as pd

# Agregar la carpeta 'src' al path
sys.path.insert(0, os.path.abspath('..'))

from src.utils.encoding_patch import apply_utf8_patch
from src.loader import DataLoader

# Aplicar parche de forma segura
apply_utf8_patch()

print("✅ Ambiente configurado correctamente y parche de codificación aplicado.")

✅ Ambiente configurado correctamente y parche de codificación aplicado.


In [2]:
# Celda 2: Ejecucion de Auditoria General
# El objeto 'report' contiene los 13 puntos de auditoria para todas las tablas
# DataLoader utiliza automaticamente las rutas definidas en config.yaml
loader = DataLoader()
report = loader.load_and_audit()

print(f"✅ ¡Extraccion completada! Se procesaron {len(report['tables'])} tablas.")

2026-02-18 13:10:33,859 - INFO - Loaded .env from c:\Users\USUARIO\Documents\Forecaster\Mi_Bunuelito\.env
2026-02-18 13:10:33,860 - INFO - Initializing Supabase client for https://ravidnmzfbpbtbnlplmh.supabase.co...
2026-02-18 13:10:34,136 - INFO - Supabase client initialized successfully.
2026-02-18 13:10:35,854 - INFO - INCREMENTAL LOAD for ventas_diarias since 2026-02-10 (Recursive Pagination)
2026-02-18 13:10:36,470 - INFO - HTTP Request: GET https://ravidnmzfbpbtbnlplmh.supabase.co/rest/v1/ventas_diarias?select=%2A&fecha=gt.2026-02-10&offset=0&limit=1000 "HTTP/2 200 OK"
2026-02-18 13:10:36,473 - INFO - No new records found for ventas_diarias.
2026-02-18 13:10:36,534 - INFO - INCREMENTAL LOAD for macro_economia since 2026-02-01 (Recursive Pagination)
2026-02-18 13:10:36,725 - INFO - HTTP Request: GET https://ravidnmzfbpbtbnlplmh.supabase.co/rest/v1/macro_economia?select=%2A&fecha=gt.2026-02-01&offset=0&limit=1000 "HTTP/2 200 OK"
2026-02-18 13:10:36,728 - INFO - No new records found

✅ ¡Extraccion completada! Se procesaron 4 tablas.


## 1. Integridad Estructural y Contrato
Diagnóstico sobre el cumplimiento del esquema definido y dimensiones de las tablas.

In [3]:
# Celda 3: Auditoria de Estructura (Puntos 1.1 al 1.3)
for table, audit in report["tables"].items():
    print(f"TABLA: {table.upper()}")
    s = audit["structural_integrity"]
    print(f"  - Contrato Cumplido: {'✅' if s['contract_fulfilled'] else '❌'}")
    print(f"  - Dimensiones: {s['shape']['rows']} filas x {s['shape']['columns']} columnas")
    if s['additional_columns']:
        print(f"  - Columnas extra detectadas: {s['additional_columns']}")
    print("-" * 30)

TABLA: VENTAS_DIARIAS
  - Contrato Cumplido: ✅
  - Dimensiones: 2963 filas x 12 columnas
  - Columnas extra detectadas: ['id', 'created_at']
------------------------------
TABLA: MACRO_ECONOMIA
  - Contrato Cumplido: ✅
  - Dimensiones: 98 filas x 8 columnas
  - Columnas extra detectadas: ['id', 'created_at']
------------------------------
TABLA: PROMOCION_DIA
  - Contrato Cumplido: ✅
  - Dimensiones: 2963 filas x 4 columnas
  - Columnas extra detectadas: ['id', 'created_at']
------------------------------
TABLA: REDES_SOCIALES
  - Contrato Cumplido: ✅
  - Dimensiones: 2963 filas x 7 columnas
  - Columnas extra detectadas: ['id', 'created_at']
------------------------------


## 2. Calidad de Datos y Limpieza
Identificación de vacíos, duplicados y valores centinela.

In [4]:
# Celda 4: Auditoria de Calidad (Puntos 2.1 al 2.6)
for table, audit in report["tables"].items():
    print(f"TABLA: {table.upper()}")
    q = audit["data_quality"]
    print(f"  - Filas duplicadas: {q['duplicate_rows_count']}")
    print(f"  - Fechas duplicadas (con datos unicos): {q['duplicate_dates_count']}")
    
    # Nulos
    nulls = {k: v for k, v in q['null_counts'].items() if v > 0}
    print(f"  - Columnas con nulos: {nulls if nulls else 'Ninguna'}")
    
    # Centinelas
    sentinels = {k: v for k, v in q['sentinel_counts'].items() if v > 0}
    print(f"  - Valores centinela (atpicos de sistema): {sentinels if sentinels else 'Ninguno'}")
    
    if q['zero_variance_cols']:
        print(f"  - Columnas sin varianza (constantes): {q['zero_variance_cols']}")
    
    print("-" * 30)

TABLA: VENTAS_DIARIAS
  - Filas duplicadas: 0
  - Fechas duplicadas (con datos unicos): 0
  - Columnas con nulos: Ninguna
  - Valores centinela (atpicos de sistema): Ninguno
------------------------------
TABLA: MACRO_ECONOMIA
  - Filas duplicadas: 0
  - Fechas duplicadas (con datos unicos): 0
  - Columnas con nulos: Ninguna
  - Valores centinela (atpicos de sistema): Ninguno
  - Columnas sin varianza (constantes): ['created_at']
------------------------------
TABLA: PROMOCION_DIA
  - Filas duplicadas: 0
  - Fechas duplicadas (con datos unicos): 0
  - Columnas con nulos: Ninguna
  - Valores centinela (atpicos de sistema): Ninguno
------------------------------
TABLA: REDES_SOCIALES
  - Filas duplicadas: 0
  - Fechas duplicadas (con datos unicos): 0
  - Columnas con nulos: Ninguna
  - Valores centinela (atpicos de sistema): Ninguno
------------------------------


## 3. Salud de la Serie de Tiempo
Análisis de la continuidad temporal y rangos de fechas.

In [5]:
# Celda 5: Auditoria Temporal (Puntos 3.1 al 3.2)
for table, audit in report["tables"].items():
    print(f"TABLA: {table.upper()}")
    t = audit["time_series_health"]
    if "date_range" in t:
        print(f"  - Rango: {t['date_range']['min']} hasta {t['date_range']['max']}")
        print(f"  - Huecos detectados (Gaps): {'❌ ' + str(t['gaps_detected_count']) if t['has_gaps'] else '✅ Ninguno'}")
    else:
        print("  - No se detecto columna de fecha para esta tabla.")
    print("-" * 30)

TABLA: VENTAS_DIARIAS
  - Rango: 2018-01-01 hasta 2026-02-10
  - Huecos detectados (Gaps): ✅ Ninguno
------------------------------
TABLA: MACRO_ECONOMIA
  - Rango: 2018-01-01 hasta 2026-02-01
  - Huecos detectados (Gaps): ❌ 2856
------------------------------
TABLA: PROMOCION_DIA
  - Rango: 2018-01-01 hasta 2026-02-10
  - Huecos detectados (Gaps): ✅ Ninguno
------------------------------
TABLA: REDES_SOCIALES
  - Rango: 2018-01-01 hasta 2026-02-10
  - Huecos detectados (Gaps): ✅ Ninguno
------------------------------


## 4. Perfilamiento Estadistico y Anomalias
Resumen estadístico y detección de outliers técnicos.

In [6]:
# Celda 6: Perfilamiento y Outliers (Puntos 4.1 al 4.3)
for table, audit in report["tables"].items():
    print(f"TABLA: {table.upper()}")
    st = audit["statistical_profiling"]
    
    print("  - Resumen de Outliers (IQR):")
    for col, prof in st["numeric_profile"].items():
        count = prof['outliers']['count']
        status = "⚠️" if count > 0 else "✅"
        print(f"    {status} {col}: {count} outliers")
        
    print("-" * 30)

TABLA: VENTAS_DIARIAS
  - Resumen de Outliers (IQR):
    ✅ id: 0 outliers
    ⚠️ total_unidades_entregadas: 69 outliers
    ⚠️ unidades_precio_normal: 67 outliers
    ⚠️ unidades_promo_pagadas: 488 outliers
    ⚠️ unidades_promo_bonificadas: 488 outliers
    ✅ precio_unitario_full: 0 outliers
    ✅ costo_unitario: 0 outliers
    ⚠️ ingresos_totales: 133 outliers
    ⚠️ costo_total: 90 outliers
    ⚠️ utilidad: 120 outliers
------------------------------
TABLA: MACRO_ECONOMIA
  - Resumen de Outliers (IQR):
    ✅ id: 0 outliers
    ⚠️ ipc_mensual: 24 outliers
    ✅ trm_promedio: 0 outliers
    ⚠️ tasa_desempleo: 9 outliers
    ✅ costo_insumos_index: 0 outliers
    ⚠️ confianza_consumidor: 12 outliers
------------------------------
TABLA: PROMOCION_DIA
  - Resumen de Outliers (IQR):
    ✅ id: 0 outliers
    ⚠️ es_promo: 488 outliers
------------------------------
TABLA: REDES_SOCIALES
  - Resumen de Outliers (IQR):
    ✅ id: 0 outliers
    ⚠️ inversion_facebook: 568 outliers
    ⚠️ invers

## 5. Persistencia del Reporte (Regla 6.68)
Guardado del reporte de auditoría en la carpeta de experimentos para trazabilidad.

In [7]:
# Celda 7: Guardado del reporte oficial (Lab Environment)
output_dir = os.path.join("..", "experiments", "phase_01_discovery", "artifacts")
os.makedirs(output_dir, exist_ok=True)

report_path = os.path.join(output_dir, "phase_01_discovery.json")
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=4, ensure_ascii=False)

print(f"✅ Reporte guardado exitosamente en: {report_path}")

✅ Reporte guardado exitosamente en: ..\experiments\phase_01_discovery\artifacts\phase_01_discovery.json
